In [ ]:
# Convex optimization algorithms
# Author: Félix Watine
# Date: June 2023

using LinearAlgebra

# Function Derivative for f: R -> R
"""
    derivative(f, x)

Compute the numerical derivative of a function `f` at point `x`.

# Arguments
- `f::Function`: The function for which the derivative is computed.
- `x::Float64`: The point at which to compute the derivative.

# Returns
- `Float64`: The numerical derivative of `f` at `x`.
"""
function derivative(f, x)
    h = 0.001
    return (f(x + h) - f(x)) / h
end

# Gradient Descent Optimization for f: R -> R
"""
    gradient_descent(f, k, l)

Perform gradient descent to optimize function `f` with a fixed learning rate `l`.

# Arguments
- `f::Function`: The function to optimize.
- `k::Int`: Number of iterations.
- `l::Float64`: Learning rate.

# Returns
- `Float64`: The optimized value after `k` iterations.
"""
function gradient_descent(f, k, l)
    x = 0.0
    for i in 1:k
        x -= l * derivative(f, x)
    end
    return x
end

# Gradient of f: R^n -> R
"""
    gradient(f, X)

Compute the gradient of a function `f` at point `X`.

# Arguments
- `f::Function`: The function for which the gradient is computed.
- `X::Vector{Float64}`: The point at which to compute the gradient.

# Returns
- `Vector{Float64}`: The gradient of `f` at `X`.
"""
function gradient(f, X)
    X = Float64.(X)
    h = 0.0001
    n = length(X)
    grad = zeros(n)
    for i in 1:n
        X_adj = copy(X)
        X_adj[i] += h
        grad[i] = (f(X_adj) - f(X)) / h
    end
    return grad
end

# Gradient Descent Optimization for f: R^n -> R
"""
    gradient_descent_n(f, n, k, l)

Perform gradient descent for a function `f` with `n` variables.

# Arguments
- `f::Function`: The function to optimize.
- `n::Int`: Dimensionality of the input space.
- `k::Int`: Number of iterations.
- `l::Float64`: Learning rate.

# Returns
- `Vector{Float64}`: The optimized point after `k` iterations.
"""
function gradient_descent_n(f, n, k, l)
    X = zeros(n)
    for i in 1:k
        X -= l * gradient(f, X)
    end
    return X
end

# Gradient Descent with Optimal Step Size (not working)
"""
    gradient_descent_opt(f, n, k)

Perform gradient descent with an optimal step size.

# Arguments
- `f::Function`: The function to optimize.
- `n::Int`: Dimensionality of the input space.
- `k::Int`: Number of iterations.

# Returns
- `Vector{Float64}`: The optimized point after `k` iterations.
"""
function gradient_descent_opt(f, n, k)
    X = ones(n)
    for i in 1:k
        function step_size(u)
            return f(X - u * gradient(f, X))
        end
        l = gradient_descent(step_size, n, 100)
        X -= l * gradient(f, X)
    end
    return X
end

# Gradient Descent with Optimal Step Size (Armijo Rule)
"""
    gradient_descent_opt_2(f, n, k)

Perform gradient descent using the Armijo rule for step size.

# Arguments
- `f::Function`: The function to optimize.
- `n::Int`: Dimensionality of the input space.
- `k::Int`: Number of iterations.

# Returns
- `Vector{Float64}`: The optimized point after `k` iterations.
"""
function gradient_descent_opt_2(f, n, k)
    X = ones(n)
    t = 1.0
    for i in 1:k
        function h(u)
            return f(X - u * gradient(f, X))
        end
        function l(u)
            return f(X) + u * dot(gradient(f, X), -gradient(f, X))
        end
        while h(t) > l(0.5 * t)
            t /= 2
        end
        X -= t * gradient(f, X)
    end
    return X
end

# Partial Derivative of f: R^n -> R
"""
    partial_derivative(f, i, x)

Compute the partial derivative of a function `f` with respect to `i` at point `x`.

# Arguments
- `f::Function`: The function for which the partial derivative is computed.
- `i::Int`: The index of the partial derivative.
- `x::Vector{Float64}`: The point at which to compute the partial derivative.

# Returns
- `Float64`: The partial derivative of `f` with respect to `i` at `x`.
"""
function partial_derivative(f, i, x)
    x = Float64.(x)
    x_adj = copy(x)
    h = 0.0001
    x_adj[i] += h
    return (f(x_adj) - f(x)) / h
end

# Hessian Matrix of f: R^n -> R
"""
    hessian(f, x)

Compute the Hessian matrix of a function `f` at point `x`.

# Arguments
- `f::Function`: The function for which the Hessian is computed.
- `x::Vector{Float64}`: The point at which to compute the Hessian.

# Returns
- `Matrix{Float64}`: The Hessian matrix of `f` at `x`.
"""
function hessian(f, x)
    n = length(x)
    x = Float64.(x)
    H = zeros(n, n)
    for i in 1:n
        for j in 1:n
            function first(x)
                return partial_derivative(f, j, x)
            end
            H[j, i] = partial_derivative(first, i, x)
        end
    end
    return H
end

# Pure Newton Method
"""
    newton_pure(f, n, x_0, k)

Perform the pure Newton method to optimize function `f`.

# Arguments
- `f::Function`: The function to optimize.
- `n::Int`: Dimensionality of the input space.
- `x_0::Vector{Float64}`: Initial point.
- `k::Int`: Number of iterations.

# Returns
- `Vector{Float64}`: The optimized point after `k` iterations.
"""
function newton_pure(f, n, x_0, k)
    x = x_0
    for i in 1:k
        x -= inv(hessian(f, x)) * gradient(f, x)
    end
    return x
end

# Cyclic Newton Method
"""
    newton_cyclic(f, n, x_0, k, m)

Perform the cyclic Newton method with updated Hessian.

# Arguments
- `f::Function`: The function to optimize.
- `n::Int`: Dimensionality of the input space.
- `x_0::Vector{Float64}`: Initial point.
- `k::Int`: Number of iterations.
- `m::Int`: Update frequency for the Hessian.

# Returns
- `Vector{Float64}`: The optimized point after `k` iterations.
"""
function newton_cyclic(f, n, x_0, k, m)
    x = x_0
    H_inv = inv(hessian(f, x))
    for i in 1:k
        x -= H_inv * gradient(f, x)
        if i % m == 0
            H_inv = inv(hessian(f, x))
        end
    end
    return x
end

# Quasi-Newton SR1 Method
"""
    newton_SR1(f, n, x_0, k)

Perform the SR1 quasi-Newton method for optimization.

# Arguments
- `f::Function`: The function to optimize.
- `n::Int`: Dimensionality of the input space.
- `x_0::Vector{Float64}`: Initial point.
- `k::Int`: Number of iterations.

# Returns
- `Vector{Float64}`: The optimized point after `k` iterations.
"""
function newton_SR1(f, n, x_0, k)
    x = x_0
    S = Diagonal(ones(n))
    for i in 1:k
        x_prev = x
        x -= S * gradient(f, x_prev)
        delta = x - x_prev
        gamma = gradient(f, x) - gradient(f, x_prev)
        v = delta - S * gamma
        denominator = dot(v, gamma)
        if abs(denominator) < 1e-8
            break
        else
            S += (v * v') / denominator
        end
    end
    return x
end

# Example Function to Optimize
"""
    f_2(X)

Example quadratic function to optimize.

# Arguments
- `X::Vector{Float64}`: The input vector.

# Returns
- `Float64`: The function value at `X`.
"""
function f_2(X)
    A = [4 1 2; 1 5 3; 2 3 6]
    B = [1, 1, 1]
    return 0.5 * (X' * A * X) + B' * X
end

# Example Usage
A = [4 1 2; 1 5 3; 2 3 6]
B = [1 1 1]
x_0 = zeros(3)

@time println("Optimal X: ", inv(A) * B')
@time println("X after gradient descent: ", gradient_descent_n(f_2, 3, 10, 0.1))
@time println("X after gradient descent with Armijo rule: ", gradient_descent_opt_2(f_2, 3, 10))
@time println("X after pure Newton method: ", newton_pure(f_2, 3, x_0, 10))
@time println("X after cyclic Newton method: ", newton_cyclic(f_2, 3, x_0, 100, 10))
@time println("X after SR1 quasi-Newton method: ", newton_SR1(f_2, 3, x_0, 10))
